# Análisis Semántico Latente
En este cuaderno se realizará la implementación del modelo de Analisis Semántico Latente (LSA) para dos documentos de prueba dentro del corpus.

Esta etapa esta compuesta de las siguientes fases:
1. Cargar datos.
2. Tokenizar por frases el corpus.
3. Aplicar el preprocesamiento al nuevo corpus.
4. Crear diccionarios para relacionar frases e identificadores.
5. Construir la matriz de A del Análisis Semántico Latente.
6. Aplicar la técnica de Análisis Semántico Latente.
7. Generación de resumen
8. Evaluar el resumen obtenido

In [1]:
#Importar elementos necesarios de las librerías
import os, shutil, re, pickle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from rouge import Rouge
from nltk.corpus import PlaintextCorpusReader
from nltk.stem.snowball import SpanishStemmer
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [2]:
#Funciones auxiliares

def tokenize_sentence(path, file_name):
    #Retorna un documento tokenizado por frases
    doc = []
    text = PlaintextCorpusReader(path, file_name)
    paragraphs = text.paras()
    for paragraph in paragraphs:
        for sentence in paragraph:
            low, i = 0,0
            while i < len(sentence):
                token = sentence[i].split('.')
                if len(token)-1:
                    doc.append(sentence[low:i])
                    low=i+1
                    i+=2
                else:
                    i+=1
            if low!=i-1:
                doc.append(sentence[low:i])
    return doc

def preprocess(doc, stopwords, stemmer):
    #Aplica el preprocesamiento establecido
    #Adicionalmente, retorna el documento original sin las filas vacías por el preprocesamiento 
    doc_preprocesed, doc_reduced = [], []
    for original_sentence in doc:
        preprocessed_sentence = []
        for token in original_sentence:
            if stemmer.stem(token) not in stopwords:
                preprocessed_sentence.append(stemmer.stem(token))
        if len(preprocessed_sentence) and preprocessed_sentence not in doc_preprocesed:
            doc_preprocesed.append(preprocessed_sentence)
            doc_reduced.append(original_sentence)
    return doc_preprocesed, doc_reduced

def get_dictionaries(doc):
    #Retorna un par de diccionarios que relacionan una frase con un id, y un id con una frase.
    sentence2id, id2sentence = {},{}
    n_sentences = len(doc)
    for i in range(n_sentences):
        sentence = ' '.join(doc[i])
        if sentence not in sentence2id:
            sentence2id[sentence] = i
            id2sentence[i] = sentence
    return sentence2id, id2sentence

def getid2token(token2id):
    #Retorna un diccionario de tokens a id, a partir de un diccionario de id a tokens
    id2token ={}
    for k,v in token2id.items():
        id2token[v] = k
    return id2token

def build_A_Matrix(document, tf_idf, token2id, doc_id):
    #Construye la matriz A que recibe el modelo de LSA como entrada
    data,row_index,col_index = [],[],[]
    tf_idf = tf_idf.toarray()
    n,m = len(document), len(token2id)
    for i in range(n):
        sentence = document[i]
        j = 0
        for token in sentence:
            if token in token2id and tf_idf[doc_id,token2id[token]] != 0:
                if (j==0) or (j>0 and token2id[token] not in col_index[-j:]):
                    value = tf_idf[doc_id,token2id[token]]
                    tf_idf_value = value
                    data.append(tf_idf_value)
                    row_index.append(i)
                    col_index.append(token2id[token])
                    j+=1
    data = np.array(data)
    row_index = np.array(row_index)
    col_index = np.array(col_index)
    A_matrix = csr_matrix((data,(row_index,col_index)),shape=(n, m),dtype=np.float64)
    return A_matrix

def generate_summary(lsa, n_sentences, sentence2id):
    #Genera un resumen con n_sentences frases
    total_sentences = lsa.shape[0]
    assert n_sentences < total_sentences
    columns = ["topic {}".format(i) for i in range(n_sentences)]
    df = pd.DataFrame(lsa,columns=columns)
    df['sentence'] = sentence2id.keys()
    summary = []
    for i in range(n_sentences):
        df = df.sort_values(by='topic {}'.format(i), ascending = False)
        j = 0
        while j < total_sentences:
            sentence = df.iloc[j]['sentence']
            if sentence not in summary:
                summary.append(sentence)
                j=total_sentences
            else:
                j+=1
    return summary

def read_reference(path, filename):
    text = PlaintextCorpusReader(path, filename)
    text = text.paras()
    flat = []
    for sentences in text:
        for sentence in sentences:
            flat_sentence = ' '.join(sentence)
            flat.append(flat_sentence)
    reference = ' '.join(flat)
    return reference

## Fase 1. Cargar datos.
En esta fase se cargarán datos obtenidos durante el preprocesamientoy que utilizaremos para obtener el modelo de LSA.

In [3]:
#Cargar el modelo de tf-idf obtenido en preprocesamiento.
filename = 'tf-idf_model.pkl'
tf_idf = pickle.load(open(filename, 'rb'))

#Cargar la matriz de tf-idf obtenido en preprocesamiento.
filename = 'tf-idf_matrix.pkl'
tf_idf_matrix = pickle.load(open(filename, 'rb'))

#Cargar la matriz de tf-idf obtenido en preprocesamiento.
filename = 'stopwords.pkl'
stopwords = pickle.load(open(filename, 'rb'))

## Fase 2. Tokenizar las frases del corpus.
En esta fase se tokenizará por el corpus por frases.

In [4]:
ruta = "D:/Documents/Documentos Universidad/Noveno/Proyecto de grado/textos"
stemmer = SpanishStemmer()
bolivar = tokenize_sentence(ruta, 'segurosbolivar-privacidad.txt')
mozilla = tokenize_sentence(ruta, 'mozilla-privacidad.txt')

## Fase 3. Aplicar el preprocesamiento al nuevo corpus.
En esta fase se aplicará el preprocesamiento establecido

In [5]:
bolivar_preprocesado, bolivar = preprocess(bolivar, stopwords, stemmer)
mozilla_preprocesado, mozilla = preprocess(mozilla, stopwords, stemmer)

## Fase 4. Crear diccionarios para relacionar frases e identificadores.
Con el fin de obtener un resumen luego de aplicar la técnica de LSA, se necesita relacionar una frase con un identificador único. Esta fase cumple con dicho objetivo.

In [6]:
frase2id_bolivar, id2frase_bolivar = get_dictionaries(bolivar)
frase2id_mozilla, id2frase_mozilla = get_dictionaries(mozilla)

## Fase 5. Construir la matriz de A del Análisis Semántico Latente.
Hasta el momento, ya tenemos una lista de frases preprocesadas, ahora lo siguiente que tenemos que hacer es construir una matriz A de NxM, con N frases y M tokens. Para esto nos apoyaremos en el TF-IDF del preprocesamiento.

In [7]:
#Diccionario que relaciona tokens con un identificador único
token2id = tf_idf.vocabulary_

#Lista de textos dentro del corpus
textos = os.listdir(ruta)

matriz_A_bolivar = build_A_Matrix(bolivar_preprocesado, tf_idf_matrix, token2id, textos.index('segurosbolivar-privacidad.txt'))
matriz_A_mozilla = build_A_Matrix(mozilla_preprocesado, tf_idf_matrix, token2id, textos.index('mozilla-privacidad.txt'))

## Fase 6. Aplicar la técnica de Análisis Semántico Latente.
Una vez construida la matriz A, se utilizará la técnica de LSA para obtener una matriz de MxK, con M frases y K temas.

In [8]:
k = 5

#Instancia un modelo de LSA
lsa_model = TruncatedSVD(n_components=k)

#Aplica el LSA a la matriz tf-idf
lsa_bolivar = lsa_model.fit_transform(matriz_A_bolivar)
lsa_mozilla = lsa_model.fit_transform(matriz_A_mozilla)

In [9]:
#Dimensión de la matriz de Seguros Bolivar
lsa_bolivar.shape

(55, 5)

In [10]:
#Dimensión de la matriz de Mozilla
lsa_mozilla.shape

(58, 5)

## Fase 7. Generación del resumen
A partir del modelo de LSA obtenido en la fase anterior, se genera un resúmen escogiendo una frase por cada tema obtenido dentro del modelo de LSA. 

In [11]:
resumen_bolivar = generate_summary(lsa_bolivar,k,frase2id_bolivar)
resumen_bolivar

['LAS COMPAÑÍAS han identificado los datos que administran , así como las actividades que desarrollan con dichos datos , en particular su recepción , conservación , disposición para los fines propios del contrato y desarrollo de actividades complementarias referidas a la promoción y el mercadeo de sus productos y servicios , así como los ofrecidos por las Compañías que hacen parte del Grupo Bolívar , al cual éstas pertenecen y así lo han indicado en las Políticas del Tratamiento disponibles para su consulta en la página web www',
 'LAS COMPAÑÍAS hacen parte del Grupo Bolívar , que tiene como matriz a Grupo Bolívar S',
 'Declaro que , con base en dicho conocimiento , autorizo para que LAS COMPAÑÍAS compartan con dichas empresas y entidades la información personal de contacto y la que sea relevante para las finalidades aquí previstas , que he suministrado',
 'Finalidades previstas para los datos obtenidos de candidatos , empleados y proveedores :',
 'Solicitar prueba de la autorización o

In [12]:
resumen_mozilla = generate_summary(lsa_mozilla,k,frase2id_mozilla)
resumen_mozilla

['Política de Privacidad de Mozilla 9 de septiembre de 2020 ¿ Qué queremos decir con " informaciones personales "?',
 'Mozilla Corporation Attn : Mozilla - Privacy 2 Harrison St',
 'Haz clic aquí para solicitar acceso a los datos personales',
 'Eventualmente divulgamos informaciones para mejorar nuestros productos y promover una Internet abierta , pero cuando lo hacemos , excluimos sus informaciones personales e intentamos divulgarlas de forma de minimizar el riesgo de identificación de los usuarios',
 'Sus informaciones sólo son divulgadas de esa manera cuando creemos de buena fe que se hace necesario para proteger los derechos , la propiedad o la seguridad del usuario en cuestión , de nuestros otros usuarios , de Mozilla o del público']

## Evaluar puntajes ROUGE

In [13]:
ruta_referencia = "D:/Documents/Documentos Universidad/Noveno/Proyecto de grado/referencias"

referencia_bolivar = read_reference(ruta_referencia, 'segurosbolivar-privacidad.txt')
resumen_bolivar = ' '.join(resumen_bolivar)
rouge = Rouge()
rouge.get_scores(resumen_bolivar, referencia_bolivar)

[{'rouge-1': {'f': 0.323450129773832,
   'p': 0.3157894736842105,
   'r': 0.3314917127071823},
  'rouge-2': {'f': 0.059620591208936896,
   'p': 0.0582010582010582,
   'r': 0.06111111111111111},
  'rouge-l': {'f': 0.205607471637261,
   'p': 0.20952380952380953,
   'r': 0.2018348623853211}}]

In [21]:
referencia_bolivar.paras()

[[['Las', 'compañias', 'pertenecientes', 'al', 'grupo', 'de', 'seguros', 'bolivar', 'almacenaran', 'y', 'manipularan', 'los', 'datos', 'recolectados', 'con', 'el', 'fin', 'de', 'promocionar', 'sus', 'productos', 'y', 'servicios', '.']], [['La', 'información', 'almacenada', 'relativa', 'al', 'estado', 'de', 'salud', 'de', 'los', 'asegurados', 'estará', 'regulada', 'por', 'los', 'estandares', 'especiales', 'de', 'la', 'expedición', 'de', 'polizas', 'de', 'salud', 'o', 'vida', '.']], ...]